## Background


We often have to compare parameter estimates across several versions of a model.

- Same model estimated with several estimators (ols, iv, gmm, ...)
- Model estimated by numerical optimization with different optimizers and or different start values for the optimization
- Monte Carlo exercises

Especially in large models (100 or more parameters) this is time consuming. Therefore, we need a plot that makes it easier to see:

- how large the differences in parameter estimates are between different models
- whether the confidence intervals of one model contain the parameter estimates of other models

If we just plot every parameter estimate and every confidence interval at the same time, the plot won't show anything because it is too full. Therefore, we need an interactive plot that always shows exactly what we want to see. Probably the best library to do this is [bokeh](https://bokeh.pydata.org/en/latest/index.html), but if you find another one you can also use it. My first strategy would be to use something like [this](https://bokeh.pydata.org/en/latest/docs/gallery/elements.html) to plot the estimates for one parameter across models. The official [tutorials](https://hub.mybinder.org/user/bokeh-bokeh-notebooks-1rrayuuy/notebooks/tutorial/00%20-%20Introduction%20and%20Setup.ipynb) also explain how to make those plots interactive, how to stack several plots into one figure and how to link the subplots within one figure.

You should start by writing a very basic function that only takes a data dictionary as argument and produces the plot I described in our last meeting. Later we will add more arguments for colors, background styles, etc. 

Please just work in this notebook.

In [1]:
# Interface
def comparison_plot(data_dict):
    """Make a comparison plot.
    
    Args:
        data_dict (dict): The keys are the names of different models.
            The values are pd.DataFrames where the index contains the names of 
            the parameters and there are three columns:
                - 'params', containing the point estimates
                - 'lower', containing the lower bound of the confidence interval
                - 'upper', containing the upper bound of the confidence interval
        
    """

## Tasks

1. Define two or three different data dictionaries that differ in the following dimensions
    - longer and shorter parameter names
    - some parameters have a large variance across models, some have a small variance
    - some have wide confidence intervals (larger than the variance of the parameter value), some have a small one.
2. Define the basic plot where based on [this](https://bokeh.pydata.org/en/latest/docs/gallery/elements.html). When hovering over a point, the pop-up window should display:
    - the exact parameter value
    - the lower and upper bound of the confidence interval from that model
    - the model name
    - the standard deviation of this parameter across models
3. Implement the clicking action, i.e. when clicking on a point, the points that belong to this model are highlighted (acrross all subplots) while all other points get more opaque. This requires linking the different subplots. 
4. We meet again.

If you have questions, just ask me again and we can meet.

In [2]:
####################

Dictionaries

In [1]:
import pandas as pd
import numpy as np

m_1 = pd.DataFrame(np.array([[2, 1.5, 3], [0.1, 0.001, 0.15], [30, 20, 40]]), columns = ['params', 'lower', 'upper'], 
                   index = ['p_1', 'p_2', 'p_3'])
m_2 = pd.DataFrame(np.array([[1, 1, 3], [0.03, 0, 0.09], [26, 20, 30]]), columns = ['params', 'lower', 'upper'], 
                   index = ['p_1', 'p_2', 'p_3'])
m_3 = pd.DataFrame(np.array([[3.5, 3.3, 5], [0.2, 0.05, 0.5], [27, 26, 36]]), columns = ['params', 'lower', 'upper'], 
                   index = ['p_1', 'p_2', 'p_3'])
m_4 = pd.DataFrame(np.array([[1.3, 1, 2], [0.14, 0.001, 0.2], [35, 25, 40]]), columns = ['params', 'lower', 'upper'], 
                   index = ['p_1', 'p_2', 'p_3'])

model_1 = pd.DataFrame(np.array([[2, 1.5, 3], [0.1, 0.001, 0.15], [30, 20, 40]]), columns = ['params', 'lower', 'upper'], 
                       index = ['p_1', 'p_2', 'p_3'])
model_2 = pd.DataFrame(np.array([[0.5, 0, 2.2], [0.01, 0, 0.09], [29, 20, 30]]), columns = ['params', 'lower', 'upper'], 
                       index = ['p_1', 'p_2', 'p_3'])
model_3 = pd.DataFrame(np.array([[3.7, 3.3, 5], [0.3, 0.05, 0.5], [27, 26, 36]]), columns = ['params', 'lower', 'upper'], 
                       index = ['p_1', 'p_2', 'p_3'])
model_4 = pd.DataFrame(np.array([[1.3, 1, 2], [0.14, 0.001, 0.2], [32, 25, 40]]), columns = ['params', 'lower', 'upper'], 
                       index = ['p_1', 'p_2', 'p_3'])

mo_1 = pd.DataFrame(np.array([[2.8, 1.5, 3], [0.1, 0.001, 0.15], [28, 20, 40]]), columns = ['params', 'lower', 'upper'], 
                    index = ['p_1', 'p_2', 'p_3'])
mo_2 = pd.DataFrame(np.array([[2.9, 1, 3], [0.09, 0, 0.09], [26, 20, 30]]), columns = ['params', 'lower', 'upper'], 
                    index = ['p_1', 'p_2', 'p_3'])
mo_3 = pd.DataFrame(np.array([[3.3, 3.3, 5], [0.14, 0.05, 0.5], [27, 26, 36]]), columns = ['params', 'lower', 'upper'], 
                    index = ['p_1', 'p_2', 'p_3'])
mo_4 = pd.DataFrame(np.array([[1.8, 1, 2.3], [0.12, 0.001, 0.2], [26, 25, 40]]), columns = ['params', 'lower', 'upper'], 
                    index = ['p_1', 'p_2', 'p_3'])

dic_1 = {'model_1' : m_1, 'model_2' : m_2, 'model_3' : m_3, 'model_4' : m_4}
dic_2 = {'model_1' : model_1, 'model_2' : model_2, 'model_3' : model_3, 'model_4' : model_4}
dic_3 = {'model_1' : mo_1, 'model_2' : mo_2, 'model_3' : mo_3, 'model_4' : mo_4}

Function

In [6]:
def comparison_plot(data_dict):
    """Make a comparison plot.
    
    Args:
        data_dict (dict): The keys are the names of different models.
            The values are pd.DataFrames where the index contains the names of 
            the parameters and there are three columns:
                - 'params', containing the point estimates
                - 'lower', containing the lower bound of the confidence interval
                - 'upper', containing the upper bound of the confidence interval
        
    """
    from bokeh.models import ColumnDataSource, HoverTool, Circle
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.layouts import gridplot, column
    
    #Make new DataFrame to link graphs with ColumnDataSource

    dic = data_dict.copy()

    for value in dic.values():
        M = len(value.index)

    N = len(dic)

    params = []
    lower = []
    upper = []
    parameter_lists = [ [] for i in range(M) ]
    param_name = []
    stds_list = []
    model = []

    for key, value in dic.items():
        model.append(key)
        param_name = value.index
        params.append(value["params"])
        lower.append(value["lower"])
        upper.append(value["upper"])
        for i in range(M):
            parameter_lists[i].append(value["params"][i])

    for i in range(M):
        stds_list.append(np.std(parameter_lists[i]))

    params = np.array(params)
    params = params.ravel()
    lower = np.array(lower)
    lower = lower.ravel()
    upper = np.array(upper)
    upper = upper.ravel()
    param_name = list(param_name)
    models = np.repeat(model, M)
    param_names = np.array(param_name*N)
    stds = np.array(stds_list*N)

    s = pd.DataFrame({'params' : params, 'stds' : stds, 'lower' : lower, 'upper' : upper, 
                      'models' : models, 'param_names' : param_names})
    
    #Make plot
    options = dict(plot_width=600, plot_height=300, tools="pan,wheel_zoom,box_zoom,box_select,tap,reset,save")
    source = ColumnDataSource(s)
    output_notebook()

    #Make plot with model and parameter names
    p0 = figure(title= "Models", x_range=model, y_range=param_name, toolbar_location = "right", plot_width=600, plot_height=150, 
                tools="pan,wheel_zoom,box_zoom,box_select,tap,reset,save")
    p0.grid.grid_line_alpha = 0
    p0.circle('models', 'param_names', fill_color="salmon", size=8, source=source)
    
    #Make plot with estimates and parameter names
    TOOLTIPS = [("parameter value", "@params"),
                ("confidence interval", "(@lower{(0.000)}, @upper{(0.000)})"),
                ("model", "@models"),
                ("standard deviation", "@stds{(0.0000)}")]

    p1 = figure(title= "Estimation Comparison of parameter 1", y_range = param_name, toolbar_location = "right", **options)
    p1.hbar('param_names', left='lower', right='upper', height=0.3, fill_alpha=0.2, nonselection_fill_alpha=0, 
            line_color=None, source=source)
    circle_glyph = p1.circle('params', 'param_names', fill_color="blue", selection_fill_color="green", 
                             nonselection_fill_alpha=0.2, size=8, source=source)
    hover = HoverTool(renderers = [circle_glyph], tooltips=TOOLTIPS)
    p1.tools.append(hover)
    
    #Make gridplot with both plots
    p = gridplot([p0, p1], toolbar_location="right", ncols=1)

    return show(p)

In [7]:
plot_dict_1 = comparison_plot(dic_1)

Loading BokehJS ...

In [4]:
plot_dict_2 = comparison_plot(dic_2)

Loading BokehJS ...

In [5]:
plot_dict_3 = comparison_plot(dic_3)

Loading BokehJS ...